# Angepasster Code zur Verarbeitung der ISCRIC Daten zu den CAMELS-DE Daten über Bodenzusammensetzung

<!-- Download der IRIC Daten -->

In [ ]:
import os
import rasterio
from owslib.wcs import WebCoverageService
from catchments import get_catchment_gdf

Festlegung der Gebiets-ID

In [ ]:
ID = 1

output_dir = ".../input_data/isric"

Download der ISRIC-Daten

In [ ]:
catchment_gdf = get_catchment_gdf(ID)

# Transform the catchment to the ISRIC projection
crs_isric = 'PROJCS["Interrupted_Goode_Homolosine",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["Degree",0.0174532925199433]],PROJECTION["Interrupted_Goode_Homolosine"],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]'
catchment_gdf = catchment_gdf.to_crs(crs_isric)

# Download the ISRIC data
variables = ["sand", "silt", "clay"]
min_x, min_y, max_x, max_y = catchment_gdf.total_bounds
buffer_width, buffer_height = (max_x - min_x) * 0.1, (max_y - min_y) * 0.1
min_x, min_y, max_x, max_y = min_x - buffer_width, min_y - buffer_height, max_x + buffer_width, max_y + buffer_height
subsets = [('X', float(min_x), float(max_x)), ('Y', float(min_y), float(max_y))]


for variable in variables:
    depths = ["0-5cm", "5-15cm", "15-30cm"]
    depths_no_download = []

    for depth in depths:
        path = os.path.join(output_dir, variable, f"{variable}_{depth}_mean.tiff")
        if os.path.exists(path):
            with rasterio.open(path) as isric:
                if min_x >= isric.bounds.left and max_x <= isric.bounds.right and min_y >= isric.bounds.bottom and max_y <= isric.bounds.top:
                    depths_no_download.append(depth)
                else:
                    isric.close()
                    os.remove(path)
                    print(f"Removed existing file {path} as it does not cover the input catchments.")
            isric.close()

    depths = [depth for depth in depths if depth not in depths_no_download]

    if depths_no_download:
        print(f"{variable} --- Data already exists and covers the input catchments, skipping download of {[f'{variable}_{depth}.tiff' for depth in depths_no_download]}.")

    if not depths:
        continue

    wcs = WebCoverageService(f"http://maps.isric.org/mapserv?map=/map/{variable}.map", version="2.0.1")
    coverage_ids = [content for content in wcs.contents if variable in content and "mean" in content]
    coverage_ids = [coverage_id for coverage_id in coverage_ids if any(depth in coverage_id for depth in depths)]

    variable_dir = os.path.join(output_dir, variable)
    if not os.path.exists(variable_dir):
        os.makedirs(variable_dir)

    for coverage_id in coverage_ids:            
        try:
            response = wcs.getCoverage(
                identifier=[coverage_id], 
                crs="EPSG:4326",  
                subsets=subsets, 
                resx=250, resy=250, 
                format="image/tiff"
            )
            with open(os.path.join(variable_dir, f"{coverage_id}.tiff"), "wb") as file:
                file.write(response.read())
            print(f"Downloaded {coverage_id}.tiff")
        except Exception as e:
            print(f"Failed to download {coverage_id}: {e}")

os.system(f"chmod -R 777 {output_dir}")

Nachverarbeitung der Daten
(extract_iscric.R wurde in Python Skript umgewandelt)

In [ ]:
import numpy as np
from rasterstats import zonal_stats
import geopandas as gpd

# Load the catchments
catchments = catchment_gdf

# List of variables
variables = ["sand", "silt", "clay"]

# Choose the depths
depths = ["0-5cm", "5-15cm", "15-30cm"]

# Loop over the variables
for variable in variables:
    print(f"Start processing variable {variable}...")

    # Loop over the depths 
    for depth in depths:
        filename = f".../input_data/isric/{variable}/{variable}_{depth}_mean.tiff"
        with rasterio.open(filename) as src:
            isric = src.read(1)
            affine = src.transform
            crs = src.crs

        catchments = catchments.to_crs(crs)

        # Statistics
        stats = ["mean", "min", "max"]

        # Extracting the raster data for all catchments
        print(f"Extraktion der Rasterdaten für alle Catchments für Variable {variable} und Tiefe {depth}...")
        extracted_rast = zonal_stats(catchments, isric, stats=stats, affine=affine, prefix="stat_")
        extracted_df = gpd.GeoDataFrame(extracted_rast)

        print(extracted_df)

        # Create a directory for the extracted data in case it does not exist
        os.makedirs(f".../input_data/isric_extracted/{variable}", exist_ok=True)

        # Save the extracted data
        output_file = f".../input_data/isric_extracted/{variable}/isric_{variable}_{depth}_extracted.csv"
        print(f"Speichern der extrahierten Daten in {output_file}...")
        extracted_df.to_csv(output_file, index=False)


Daten sind aus den .tiffs in .csvs extrahiert und werden nun weiterverarbeitet

In [ ]:
import pandas as pd
from glob import glob

"""
Postprocess extracted ISRIC data.  
- CAMELS-DE only uses the mean values in each depth.
- Aggregate and calculate a weighted average over depths:
    - 0-30 cm: 0-5 cm (5/30), 5-15 cm (10/30), 15-30 cm (15/30)
- Convert to common units and rename the columns:
    | **Variable** | **Mapped unit** | **Conversation factor** | **Common unit**   | **CAMELS-DE variable name** |
    |--------------|-----------------|-------------------------|-------------------|-----------------------------|
    | clay         | g/kg            | 10                      | g/100g (%)        | clay                        |
    | silt         | g/kg            | 10                      | g/100g (%)        | silt                        |
    | sand         | g/kg            | 10                      | g/100g (%)        | sand                        |
"""

df_result = pd.DataFrame()

# Load the data
for variable in ["clay", "silt", "sand"]:
    # Get the files
    files = glob(f".../input_data/isric_extracted/{variable}/*.csv")

    # Create dictionary over depths with the data
    data = {os.path.basename(file).split(f"{variable}_")[1].split("_")[0]: pd.read_csv(file) for file in files}

    # Aggregate the data
    aggregated_data = {}
    # 0-30 cm
    aggregated_data["0-30cm"] = (data["0-5cm"].iloc[:,1:] * (5 / 30) + data["5-15cm"].iloc[:,1:] * (10 / 30) + data["15-30cm"].iloc[:,1:] * (15 / 30))

    # Convert to common units
    aggregated_data = {depth: df / 10 for depth, df in aggregated_data.items()}

    dfs = {}
    for depth, df in aggregated_data.items():
        # Add the depth to the column names
        df.columns = [f"{depth.replace('-', '_')}_{column}" if column != "gauge_id" else column for column in df.columns]
        # Save the dataframe
        dfs[depth] = df

    # Add the camels variable names to the column names
    for depth, df in dfs.items():
        if variable in ["clay", "silt", "sand"]:
            df.columns = [f"{variable}_{column}" if column != "gauge_id" else column for column in df.columns]

    # Concatenate the dataframes, keep only the first gauge_id column
    df_result_variable = pd.concat(dfs.values(), axis=1)

    # Add the data to the result dataframe
    df_result = pd.concat([df_result, df_result_variable], axis=1)


# only keep columns that include _mean and the gauge_id
df_result = df_result[[col for col in df_result.columns if "mean" in col or col == "gauge_id"]]

# Remove '_stat' from column names 
df_result.columns = [col.replace('_stat', '') for col in df_result.columns]

# round to 2 decimals
df_result = df_result.round(2)

df_result.insert(0, "gauge_id", ID)

Abspeichern der Daten

In [ ]:
# Save the data
file_path = f".../output_data/camels_de/CAMELS_DE_soil_attributes.csv"
if not os.path.isfile(file_path):
    df_result.to_csv(file_path, index=False)  # write header if file does not exist
else:
    df_result.to_csv(file_path, mode='a', header=False, index=False)  # append data without writing the header

print(f"Postprocessing of all variables to soil_attributes.csv finished.")